# $\textit{Analisis de respuestas impulsivas}$
$\textit{Filtro lineal optimo}$

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import scipy.signal as sp
import scipy.io as io
import scipy.linalg as lin

In [ ]:
%matplotlib widget

In [ ]:
fs = 48000

In [ ]:
t = np.linspace(0, 10, 10*fs, endpoint=False)
f0 = 20
f1 = 20000
T = 10
k = (f1/f0)**(1/T)

chirpLinearSignal = sp.chirp(t, f0=f0, t1=10, f1=f1, method='linear')

chirpExpSignal = np.sin(2 * np.pi * f0 * (k**t-1)/np.log(k))

gaussSignal = np.random.normal(0, 1, size=fs*10)

squareSignal = sp.square(2 * np.pi * 100 * t)



In [ ]:
# io.wavfile.write("chirpLinearSignal.wav", fs, chirpLinearSignal)
# io.wavfile.write("chirpExpSignal.wav", fs, chirpExpSignal)
# io.wavfile.write("noiseSignal.wav", fs, gaussSignal)
# io.wavfile.write("squareSignal.wav", fs, squareSignal)

In [ ]:
def plot_spectrogram(title, w, fs):
    ff, tt, Sxx = sp.spectrogram(w, fs=fs, nperseg=256, nfft=576)
    fig, ax = plt.subplots()
    ax.pcolormesh(tt, ff, Sxx, cmap='gray_r',
                  shading='gouraud')
    ax.set_title(title)
    ax.set_xlabel('t (sec)')
    ax.set_ylabel('Frequency (Hz)')
    ax.grid(True)


In [ ]:
fs1, musicSignal = io.wavfile.read("musicSignal.wav")
fs2, VoiceSignal = io.wavfile.read("VoiceSignal.wav")
fs3, chirpLinearSignal = io.wavfile.read("chirpLinearSignal.wav")
fs4, chirpExpSignal = io.wavfile.read("chirpExpSignal.wav")
fs5, gaussSignal = io.wavfile.read("noiseSignal.wav")
fs6, squareSignal = io.wavfile.read("squareSignal.wav")

In [ ]:
fs1, recmusicSignal = io.wavfile.read("recmusicSignal.wav")
fs2, recVoiceSignal = io.wavfile.read("recVoiceSignal.wav")
fs3, recchirpLinearSignal = io.wavfile.read("recchirpLinearSignal.wav")
fs4, recchirpExpSignal = io.wavfile.read("recchirpExpSignal.wav")
fs5, recgaussSignal = io.wavfile.read("recnoiseSignal.wav")
fs6, recsquareSignal = io.wavfile.read("recsquareSignal.wav")

recmusicSignal = (recmusicSignal.T)[1]
recVoiceSignal = (recVoiceSignal.T)[1]
recchirpLinearSignal = (recchirpLinearSignal.T)[1]
recchirpExpSignal = (recchirpExpSignal.T)[1]
recgaussSignal = (recgaussSignal.T)[1]
recsquareSignal = (recsquareSignal.T)[1]


In [ ]:
orgsignal = {"music": musicSignal, "voice": VoiceSignal, "chirpLin": chirpLinearSignal,
          "chirpExp": chirpExpSignal, "gauss": gaussSignal, "square": squareSignal}
recsignal = {"music": recmusicSignal, "voice": recVoiceSignal, "chirpLin": recchirpLinearSignal,
          "chirpExp": recchirpExpSignal, "gauss": recgaussSignal, "square": recsquareSignal}

In [ ]:
ipd.Audio(squareSignal, rate=fs)

Comparamos los espectros de potencia

In [ ]:
plt.figure(figsize=(10, 15))

for i, (name, s) in enumerate(zip(orgsignal.keys(), orgsignal.values())):
    f, Pxx_den = sp.welch(s, fs=fs, window='bartlett')
    plt.subplot(3, 2, i+1)
    plt.title(f"{name}")
    plt.xlabel('Freq [Hz]')
    plt.ylabel('PSD')
    plt.loglog(f, Pxx_den)

Viendo los espectrogramas anteriores, intuimos que la señal que mejor va funcionar para obtener h(n) es el barrido lineal porque excita al sistema (en este caso en el living de una casa) con un espectro de potencia mas constante que los demas. Este espectro plano permite que el resultado del filtro optimo ajuste directamente sobre el sistema q se quiere modelar y no tambien tome las caracteristicas de la señal que se esta utilizando. El ruido gaussiano tambien tiene una caracteristica plana, pero al ser una muestra de un ensamble no tiene un espectro realmente plano

# Modelado de sistema

$$ w_o = R^{-1}*p $$

$$ R = \begin{bmatrix}
Rxx(0) & Rxx (-1) & ... & Rxx(-(M-1)) \\
Rxx(-1) & Rxx (0) & ... & Rxx(-(M-2)) \\
... & & & \\

\end{bmatrix}
$$
$$ p = \begin{bmatrix}
Rxy(1) & Rxy (2) & ... & Rxy(M) \\

\end{bmatrix}
$$

![Alt text](image.png)

In [ ]:
def get_optimal_params(x, y, M):
    N = len(x)
    r = sp.correlate(x, x)/N
    p = sp.correlate(x, y)/N
    r = r[N-1:N-1 + M]
    p = p[N-1:N-1-(M):-1]           # Correlate calcula la cross-corr r(-k), y necesitamos r(k), y esto no es par como la autocorrelacion
    wo = lin.solve_toeplitz(r, p)

    jo = np.var(y) - np.dot(p, wo)

    NMSE = jo/np.var(y)
    
    return wo, jo, NMSE


In [ ]:
wo, jo, nmse = get_optimal_params(orgsignal['music'], recsignal['music'], 12500)

out = sp.lfilter(wo, [1.0], recsignal['music'])

print(nmse)


In [ ]:
Ms = np.linspace(1000, 20000, 100, dtype=int)
jos = np.zeros(len(Ms))
for i, M in enumerate(Ms):
    wo, jo, nmse = get_optimal_params(orgsignal['music'], recsignal['music'], M)
    jos[i] = nmse

plt.figure()
plt.plot(Ms, jos)

In [ ]:
ipd.Audio(orgsignal['music'], rate=fs)

In [ ]:
ipd.Audio(recsignal['music'], rate=fs)

In [ ]:
ipd.Audio(out/np.abs(out).max(), rate=fs, normalize=False)